In [1]:
import sys
sys.path.append(r"D:\Clarifruit\cherry_stem")

from work.unet_main import *
from work.unet import unet_model_functions

from keras.callbacks import ReduceLROnPlateau,EarlyStopping

Using TensorFlow backend.


In [2]:
train_path=r'D:\Clarifruit\cherry_stem\data\raw_data\with_maskes'
dest_path=r'D:\Clarifruit\cherry_stem\data\unet_data\training'
test_path=r'D:\Clarifruit\cherry_stem\data\raw_data\images_orig'
src_path=r'D:\Clarifruit\cherry_stem\data\unet_data\training\2019-10-13_18-57-55'
weights_name =r'unet_cherry_stem.01-0.07.hdf5'


In [9]:
params_dict = dict(

    train_path=train_path,
    save_path=dest_path,
    x_folder_name='image',
    y_folder_name='label',
    weights_file_name='unet_cherry_stem.hdf5',

    data_gen_args=dict(rotation_range=180,
                       brightness_range=[0.2, 1.],
                       shear_range=10,
                       zoom_range=0.5,
                       horizontal_flip=True,
                       vertical_flip=True,
                       channel_shift_range=5,
                       fill_mode='nearest'),
    seed=78,


    optimizer='Adam',
    optimizer_params=dict(lr=1e-5),
    loss='binary_crossentropy',
    metrics=['accuracy'],
    pretrained_weights=None,

    target_size=(256, 256),
    color_mode='rgb',
    
    batch_size=10, # my GPU cant handel any more
    
    epochs=10,
    steps_per_epoch=3000,
    valdiation_split=0.2,
    validation_steps=300,
    
    tensorboard_update_freq=1000,
    weights_update_freq = 5000,
    ontop_display_threshold=0.4)

In [10]:
model = unet_model_functions.ClarifruitUnet(**params_dict)
keras_logs_path=model.set_model_for_train(params_dict=params_dict)

2019-10-13 20:14:05,992-6d  INFO     work.unet.unet_model: <- unet model with input_size=(256, 256, 3)
Found 152 images belonging to 1 classes.
Found 38 images belonging to 1 classes.
Found 152 images belonging to 1 classes.
Found 38 images belonging to 1 classes.
Found 152 images belonging to 1 classes.
Found 38 images belonging to 1 classes.
Found 152 images belonging to 1 classes.
Found 38 images belonging to 1 classes.


In [3]:
# load model
params_dict = unet_model_functions.ClarifruitUnet.load_model(src_path,weights_name)
update_dict=dict(optimizer='Adam',
                 optimizer_params=dict(lr=1e-5,decay=1e-6,amsgrad=True))
params_dict.update(update_dict)
model = unet_model_functions.ClarifruitUnet(**params_dict)
keras_logs_path=model.set_model_for_train(params_dict=params_dict)

2019-10-13 20:15:03,053-6d  INFO     work.unet.unet_model: <- unet model with input_size=(256, 256, 3)
2019-10-13 20:15:03,053-6d  INFO     work.unet.unet_model:  model loaded from pretraind_weights=D:\Clarifruit\cherry_stem\data\unet_data\training\2019-10-13_18-57-55\unet_cherry_stem.5004-0.05.hdf5
Found 152 images belonging to 1 classes.
Found 38 images belonging to 1 classes.
Found 152 images belonging to 1 classes.
Found 38 images belonging to 1 classes.
Found 152 images belonging to 1 classes.
Found 38 images belonging to 1 classes.
Found 152 images belonging to 1 classes.
Found 38 images belonging to 1 classes.


In [4]:
print(f"tensorboard --logdir={keras_logs_path}")

tensorboard --logdir=D:\Clarifruit\cherry_stem\data\unet_data\training\2019-10-13_18-57-55\keras_logs


In [ ]:
model.fit_unet()

Epoch 1/10
 526/3000 [====>.........................] - ETA: 40:19 - loss: 0.0445 - acc: 0.7331
step 05004: saving model to D:\Clarifruit\cherry_stem\data\unet_data\training\2019-10-13_18-57-55\unet_cherry_stem.5004-0.05.hdf5
1053/3000 [=========>....................] - ETA: 31:48 - loss: 0.0423 - acc: 0.7315
step 10010: saving model to D:\Clarifruit\cherry_stem\data\unet_data\training\2019-10-13_18-57-55\unet_cherry_stem.10010-0.03.hdf5
1580/3000 [==============>...............] - ETA: 23:11 - loss: 0.0412 - acc: 0.7275
step 15016: saving model to D:\Clarifruit\cherry_stem\data\unet_data\training\2019-10-13_18-57-55\unet_cherry_stem.15016-0.04.hdf5
2107/3000 [====================>.........] - ETA: 14:32 - loss: 0.0402 - acc: 0.7311
step 20022: saving model to D:\Clarifruit\cherry_stem\data\unet_data\training\2019-10-13_18-57-55\unet_cherry_stem.20022-0.05.hdf5
2634/3000 [=========================>....] - ETA: 5:54 - loss: 0.0395 - acc: 0.7326
step 25028: saving model to D:\Clarifruit\

In [5]:
model.prediction(test_path,dest_path)


2019-10-13 09:48:57,533-6d  INFO     work.unet.unet_model_functions: prediction on files from D:\Clarifruit\cherry_stem\data\raw_data\images_orig
2019-10-13 09:48:57,535-6d  INFO     work.unet.unet_model_functions: saving predictions to D:\Clarifruit\cherry_stem\data\unet_data\training\2019-10-13_03-58-48\raw_pred


'D:\\Clarifruit\\cherry_stem\\data\\unet_data\\training\\2019-10-13_03-58-48\\raw_pred'